# 1. 準備

### 1.1. 基本設定

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns #seabornない人はpip installしてね
import os
from datetime import datetime
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.dates as mdates

# カレントディレクトリを.pyと合わせるために以下を実行
if Path.cwd().name == "notebook":
    os.chdir("..")

# 設定
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 500)
pd.set_option('display.max_columns', 500)

# 浮動小数点数を小数点以下3桁で表示するように設定
pd.set_option('display.float_format', '{:.3f}'.format)

In [ ]:
# Mac Matplotlibのデフォルトフォントをヒラギノ角ゴシックに設定
plt.rcParams['font.family'] = 'Hiragino Sans'

In [ ]:
# Windows MatplotlibのデフォルトフォントをMeiryoに設定
plt.rcParams['font.family'] = 'Meiryo'

### 1.2. csv読み込み

In [ ]:
# point_history.csvの読み込み
df_point_history_sorce = pd.read_csv('data/input/point_history_cleansing.csv')

In [ ]:
# ユーザー基本情報の読み込み
df_user_base_sorce = pd.read_csv("data/input/ユーザー基本情報_2023-12-21.csv", encoding="shift-jis")

### 1.3. データクレンジング

#### 1.3.1. df_user_base(ユーザ基本情報)のクレンジング

In [ ]:
# DataFrameのコピーを作成
df_user_base = df_user_base_sorce.copy()

# 紛らわしい列名を改名
df_user_base = df_user_base.rename(columns={'登録日時': 'アプリ登録日時', '最終利用日': 'アプリ最終利用日'})

# objectをdatetimeに変更
df_user_base['アプリ登録日時'] = pd.to_datetime(df_user_base['アプリ登録日時'], errors='coerce')
df_user_base['アプリ最終利用日'] = pd.to_datetime(df_user_base['アプリ最終利用日'], errors='coerce')
df_user_base['birthday'] = pd.to_datetime(df_user_base['birthday'], errors='coerce')

# 6歳未満(1543個)と100歳以上(12個)を削除
df_user_base = df_user_base[ (df_user_base['birthday'] < pd.to_datetime('2017-01-01')) & (df_user_base['birthday'] > pd.to_datetime('1924-01-01'))]

# df_user_baseに"age"と"age_group"のカラムを追加
df_user_base['age'] = pd.Timestamp.now().year - df_user_base['birthday'].dt.year    # ageの算出・追加

# 年代の算出・追加
bins = [0, 20, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ['0-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100']
df_user_base['age_group'] = pd.cut(df_user_base['age'], bins=bins, labels=labels, right=False)

# 今回使用しない可能性が高いカラムは削除
df_user_base = df_user_base.drop(['登録日', 'カード種類', 'スーパー', '都道府県', '市区町村', '登録店舗', 'カード登録日', 'カード更新日', 'birthday'], axis=1)
df_user_base = df_user_base.drop_duplicates(subset='利用者ID', keep='first')
df_user_base = df_user_base.sort_values(by='アプリ登録日時')
df_user_base.head()
# len(df_user_base)

#### 1.3.2. df_point_history(point_history.csv)のクレンジング

In [ ]:
# DataFrameのコピーを作成
df_point_history = df_point_history_sorce.copy()

# objectをdatetimeに変更
df_point_history['use_date'] = pd.to_datetime(df_point_history['use_date'], errors='coerce')

# 今回使用しない可能性が高いカラムは削除
df_point_history = df_point_history.drop([
    'Unnamed: 0',
    'id',
    'series_id',
    'shop_id',
    'shop_name',
    'card_id',
    'リサイクル分類ID',
    'amount',
    'created_at',
    'updated_at',
    '支店ID',
    'super',
    'shop_name_1',
    'shop_id_1',
    'updated_at_1',
    'store_opening_time',
    'store_closing_time',
    'created_at_1',
    'rps_opening_time',
    'rps_closing_time',
    'store_latitude',
    'store_longitude',
    'total_amount',], axis=1)
df_point_history = df_point_history.sort_values(by='use_date')
df_point_history.head(1)

In [ ]:
#df_point_history[df_point_history['user_id']==1138]

#### 1.3.3. 分析に必要なカラムの作成

アプリ継続利用期間（アプリ最終利用日-アプリ登録日時）

In [ ]:
# 継続利用期間を計算
df_user_base['アプリ継続利用期間(月)'] = (df_user_base['アプリ最終利用日'] - df_user_base['アプリ登録日時']).dt.days / 30  # 月単位で計算
df_user_base = df_user_base[df_user_base['アプリ継続利用期間(月)'] >= 0]
df_user_base.head()
# len(df_user_base[df_user_base['継続利用期間(月)'] == 0])

継続利用期間（point_historyのuse_date列からRPS最終利用日を抽出したver.）　231228 norosen

In [ ]:
# 利用者id を抽出
user_id_all = df_user_base['利用者ID'].values

# 各利用者id に対して「RPS利用開始日」「RPS最終利用日」を抽出
first_entries_RPS = df_point_history.groupby('user_id').first().reset_index()
last_entries_RPS = df_point_history.groupby('user_id').last().reset_index()

"""
for uid in user_id_all:

    df_one_user = df_point_history[df_point_history['user_id']==uid]
    
    if len(df_one_user) != 0:

        first_use_RPS.append(df_one_user['use_date'].iloc[0])
        
        last_use_RPS.append(df_one_user['use_date'].iloc[-1])
        
    else:

        first_use_RPS.append('NaT')
        
        last_use_RPS.append('NaT')
"""

In [ ]:
# 「RPS利用開始日」「RPS最終利用日」カラムを作成

# df_user_baseに利用開始日をマージ
df_user_base = pd.merge(df_user_base, first_entries_RPS[['user_id', 'use_date']], left_on='利用者ID', right_on='user_id', how='left')
df_user_base = df_user_base.rename(columns={'use_date':'RPS利用開始日'})

# df_user_baseに最終利用日をマージ
df_user_base = pd.merge(df_user_base, last_entries_RPS[['user_id', 'use_date']], left_on='利用者ID', right_on='user_id', how='left')
df_user_base = df_user_base.rename(columns={'use_date':'RPS最終利用日'})


df_user_base['RPS利用開始日'] = pd.to_datetime(df_user_base['RPS利用開始日'], errors='coerce')
df_user_base['RPS最終利用日'] = pd.to_datetime(df_user_base['RPS最終利用日'], errors='coerce')

In [ ]:
df_user_base = df_user_base.drop(columns=['user_id_x', 'user_id_y'])

In [ ]:
# RPS継続利用期間を計算
df_user_base['RPS継続利用期間(月)'] = (df_user_base['RPS最終利用日'] - df_user_base['RPS利用開始日']).dt.days / 30  # 月単位で計算
df_user_base = df_user_base[df_user_base['RPS継続利用期間(月)'] >= 0]

In [ ]:
# 列の順番を分かりやすいように入れ替える
order_df_user_base = ['利用者ID', 'カード登録', '現在ランク', 'age', 'age_group',
                      'アプリ登録日時', 'アプリ最終利用日', 'アプリ継続利用期間(月)',                   
                      'RPS利用開始日', 'RPS最終利用日', 'RPS継続利用期間(月)']

df_user_base = df_user_base[order_df_user_base]

In [ ]:
# アプリ登録年月ごとに集計
df_user_base['アプリ登録年月'] = df_user_base['アプリ登録日時'].dt.strftime("%Y/%m")

In [ ]:
# RPS開始日を年月ごとに集計
df_user_base['RPS開始年月'] = df_user_base['RPS利用開始日'].dt.strftime("%Y/%m")

In [ ]:
# 2021年12月にアプリを登録したユーザーに絞って解析
year_month = datetime(2021,12,1).strftime("%Y/%m")

df_user_base_year_month = df_user_base[df_user_base['アプリ登録年月']==year_month]

# RPSの開始日の方が先のユーザーは別で解析するので分ける
df_user_base_year_month_app_derived = df_user_base_year_month[(df_user_base['アプリ登録日時'] - df_user_base_year_month['RPS利用開始日']).dt.days <0]  # 240107修正: >=0 から <0 に
df_user_base_year_month_RPS_derived = df_user_base_year_month[(df_user_base['アプリ登録日時'] - df_user_base_year_month['RPS利用開始日']).dt.days >=0]   # 240107修正: <0 から >=0 に

# 2. 予測

## 2.1. 予測するユーザの選定

まず、2021/12（アプリ登録ユーザ急増日）以降の日付にフィルタリング

In [ ]:
df_analyzed_user_base = df_user_base_year_month_app_derived.copy()  # .copy（）は必ずつけること

In [ ]:
# 利用者id を抽出
user_id_analyzed = df_analyzed_user_base['利用者ID'].values

# 抽出した利用者idに対応したuser_idの行だけ抽出
df_analyzed_point_history = df_point_history[df_point_history['user_id'].isin(user_id_analyzed)]
df_analyzed_point_history = df_analyzed_point_history[df_analyzed_point_history['status']==1]
df_analyzed_point_history['use_day'] = df_analyzed_point_history['use_date'].dt.strftime("%Y/%m/%d")
df_analyzed_point_history['use_day'] = pd.to_datetime(df_analyzed_point_history['use_day'], errors='coerce')

投入回数頻度が中央値あたりのユーザが適していると仮定し、調べてみる

In [ ]:
df_count = df_analyzed_point_history['user_id'].value_counts()
df_count.median()

中央値は頻度がデータ数が少なすぎるため却下  
ヒストグラムで全体を確認

In [ ]:
# Plotting the histogram
plt.figure(figsize=(10, 6))
plt.hist(df_count, bins=200, color='blue', alpha=0.7)
plt.title('Histogram of Counts')
plt.xlabel('Count')
plt.ylabel('Frequency')
plt.ylim([0,600])
plt.grid(True)
plt.show()

ユーザ数は少ないが、学習に使うデータ量を鑑みて200回程度の人の投入量を予測する。

In [ ]:
# df_count

user_id10015が200回程度データがあるため、決定

In [ ]:
# df_analyzed_point_history[df_analyzed_point_history['user_id']==10015]

一日に何度も投入するユーザがいるため、日付ごとに投入量を合計

In [ ]:
# ユーザーごとに各日にちの古紙投入量を算出・集計
group_uid_uday = df_analyzed_point_history.groupby(['user_id', 'use_day'])['amount_kg'].sum()

LSTMで学習する形式に適した形に変更

In [ ]:
# 日付の範囲を生成する
start_date = '2021-12-01'  # アプリユーザが急増した日
end_date = '2023-12-05'    # point_history.csvの最終日
date_range = pd.date_range(start=start_date, end=end_date)

# 日付をカラムに持つデータフレームに変換し、NaNを0で埋める
df_pivot = group_uid_uday.unstack(level=-1).fillna(0)

# 日付のフォーマットを変更し、必要な範囲の日付だけにフィルタリング
df_pivot = df_pivot.reindex(columns=date_range, fill_value=0)
df_pivot.columns = df_pivot.columns.strftime('%Y/%m/%d')

# データの変換（だいぶセンスのないコード）
df_pivot = df_pivot.reset_index()
df_pivot_target = df_pivot[df_pivot['user_id']==10015]
df_pivot_target['user_id'].astype(float)
df_pivot_target = df_pivot_target.transpose()
df_pivot_target = df_pivot_target.drop('user_id', axis=0)
df_pivot_target.rename(columns ={1793: 'amount'},inplace=True)
df_pivot_target
# df_pivot_target = df_pivot_target.reset_index()
# df_pivot_target.columns = ['date', 'amount']
# df_pivot_target['date'] = pd.to_datetime(df_pivot_target['date'])

## 2.2. 予測

### 2.2.1. 投入量の総量を予測

1日ごとの投入量を、累積量に変更

In [ ]:
df_pivot_target['amount'] = df_pivot_target['amount'].cumsum()
df_pivot_target

In [ ]:
# データフレームをnumpy配列に変換
data = df_pivot_target.values

# タイムステップの設定
n_steps = 5  # 例として3日間のタイムステップを設定

# データの再形成関数
def create_dataset(data, n_steps):
    X, y = [], []
    for i in range(n_steps, len(data)):
        X.append(data[i-n_steps:i])
        y.append(data[i])
    return np.array(X), np.array(y)

# データを再形成
X, y = create_dataset(data, n_steps)

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

# LSTMモデルの構築
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(50, activation='relu', input_shape=(n_steps, 1)),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# モデルの訓練（履歴を保存）
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# モデルによる予測
predictions = model.predict(X_test)

In [ ]:
# 予測精度の評価
mae = mean_absolute_error(y_test, predictions)
print(f"平均絶対誤差: {mae}")

# 学習履歴の取得
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

# MAE評価の折れ線グラフの描画
plt.figure(figsize=(12, 6))
plt.plot(epochs, loss, 'orange', label='Training MAE')
plt.plot(epochs, val_loss, 'darkblue', label='Validation MAE')
plt.title('Training and Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.ylim([0,100])
plt.legend()
# grid.on
plt.show()

In [ ]:
predictions

In [ ]:
# 実際のデータと予測値のグラフ化
plt.figure(figsize=(12, 6))

# 実際の値のプロット
plt.plot(df_pivot_target.index[n_steps:], data[n_steps:], label='Actual', marker='', linestyle='-')

# 予測値のプロット
predicted_index = df_pivot_target.index[len(y_train) + n_steps:]
plt.plot(predicted_index, predictions, label='Predicted', marker='', linestyle='-')

# X軸のフォーマットを日付に設定
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=15))  # 15日ごとに日付を表示

plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Time Series Prediction')
plt.legend()
plt.xticks(rotation=90)  # X軸のラベルを45度回転
# plt.ylim([1100,1200])
# plt.xlim(['1971,8,1','1972,8,1'])
plt.grid()
plt.show()


### 2.2.2. 1日ごとの投入量を予測

In [ ]:
# データフレームをnumpy配列に変換
data = df_pivot_target.values

# タイムステップの設定
n_steps = 3  # 例として3日間のタイムステップを設定

# データの再形成関数
def create_dataset(data, n_steps):
    X, y = [], []
    for i in range(n_steps, len(data)):
        X.append(data[i-n_steps:i])
        y.append(data[i])
    return np.array(X), np.array(y)

# データを再形成
X, y = create_dataset(data, n_steps)

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=False)

# LSTMモデルの構築
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(50, activation='relu', input_shape=(n_steps, 1)),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# モデルの訓練（履歴を保存）
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# モデルによる予測
predictions = model.predict(X_test)

In [ ]:
# 予測精度の評価
mae = mean_absolute_error(y_test, predictions)
print(f"平均絶対誤差: {mae}")

# 学習履歴の取得
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

# MAE評価の折れ線グラフの描画
plt.figure(figsize=(12, 6))
plt.plot(epochs, loss, 'orange', label='Training MAE')
plt.plot(epochs, val_loss, 'darkblue', label='Validation MAE')
plt.title('Training and Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()

In [ ]:
# 実際のデータと予測値のグラフ化
plt.figure(figsize=(12, 6))

# 実際の値のプロット
plt.plot(df_pivot_target.index[n_steps:], data[n_steps:], label='Actual', marker='', linestyle='-')

# 予測値のプロット
predicted_index = df_pivot_target.index[len(y_train) + n_steps:]
plt.plot(predicted_index, predictions, label='Predicted', marker='', linestyle='-')

# X軸のフォーマットを日付に設定
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=15))  # 15日ごとに日付を表示

plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Time Series Prediction')
plt.legend()
plt.xticks(rotation=90)  # X軸のラベルを45度回転
plt.ylim([0,2])
plt.show()


**結果**  
・予測精度は悪い  
・日付がバグった  
・間欠データって、予測に向いてないのでは？日ごとに値を蓄積した総量のデータの方がよい？  